In [3]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import re
import time
from bs4 import BeautifulSoup
import mechanicalsoup
from urllib.request import urlopen
import requests
import pandas as pd
import sqlite3

# Part I: FT1000: Europe's Fastest Growing Companies 2023

The ranking lists those European companies that achieved the highest compound annual growth rate in revenue between 2018 and 2021.

In [5]:
url = "https://www.ft.com/ft1000-2023"
page = urlopen(url)
html = page.read().decode("utf-8")

In [6]:
soup = BeautifulSoup(html, "html.parser")

In [7]:
soup.title.string

'FT 1000: the seventh annual ranking of Europe’s fastest-growing companies'

The `<th>` HTML element defines a cell as the header of a group of table cells.

In [8]:
# column names
headers = soup.find_all("th")
header_col = []
for i in range(len(headers)):
    header_col.append(headers[i].string)
print(header_col)

['Rank', 'Name', 'in 2022 ranking', 'in 2021 ranking', 'Country', 'Sector', 'Absolute Growth Rate %', 'Compound Annual Growth Rate (CAGR) %', 'Revenue 2021 (€)', 'Revenue 2018 (€)', 'Number of employees 2021', 'Number of employees 2018', 'Founding Year']


The `<td>` HTML element defines a cell of a table that contains data. It participates in the table model.

The `<tr>` HTML element defines a row of cells in a table. The row's cells can then be established using a mix of `<td>` (data cell) and `<th>` (header cell) elements.

In [9]:
df_ft1000 = pd.DataFrame(columns=header_col, index=pd.RangeIndex(start=0, stop=1000, step=1))

In [10]:
companies = soup.find_all("tr")
for i in range(1,len(companies)-1):
    for j in range(len(headers)):
        cell_value = companies[i].find_all("td")[j].string
        df_ft1000.iloc[i-1, j] = cell_value
# print(df_ft1000)

In [11]:
df_ft1000

,Rank,Name,in 2022 ranking,in 2021 ranking,Country,Sector,Absolute Growth Rate %,Compound Annual Growth Rate (CAGR) %,Revenue 2021 (€),Revenue 2018 (€),Number of employees 2021,Number of employees 2018,Founding Year
0,1,Tripledot Studios,No,No,UK,Leisure & Entertainment,"71,507.8",794.7,"83,062,822","115,197",172,18,2016
1,2,Marshmallow,No,No,UK,"Fintech, Financial Services & Insurance","43,771.2",659.8,"94,346,611","201,333",220,8,2016
2,3,WeCo,No,No,Italy,Manufacturing,"15,048.1",433.1,"61,882,913","408,518",12,2,2015
3,4,Silverstream Technologies,No,No,UK,"Professional, Scientific & Technical Services","14,488.7",426.4,"17,259,314","118,306",48,8,2010
4,5,Gift & Go,No,No,UK,IT & Software,"13,032.6",408.3,"17,344,060","123,642",13,2,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Exclaimer,No,No,UK,IT & Software,153.1,36.3,"36,932,737","13,658,593",184,79,2001
996,997,Anthesis Group,No,Yes,UK,"Professional, Scientific & Technical Services",153.0,36.3,"49,671,979","18,379,332",880,220,2013
997,998,Globe Flight,No,No,Germany,Wholesale,152.8,36.2,"33,369,282","13,199,635",26,18,2012
998,999,SME France,No,No,France,Construction & Engineering,152.7,36.2,"2,672,500","1,057,400",15,7,2013


In [342]:
# df_ft1000.to_csv("ft1000.csv")
# files.download("ft1000.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# save the result in csv file
path = "/content/drive/My Drive/Quantitative-Project/Web Scraping in Python/ft1000.csv"
with open(path, 'w', encoding='utf-8-sig') as f:
    df_ft1000.to_csv(f)

In [37]:
# from DataFrame to SQL
conn = sqlite3.connect('ft1000_database')
c = conn.cursor()

In [38]:
c.execute('CREATE TABLE IF NOT EXISTS ft1000 (Rank number, Name text, in_2022_ranking text, in_2021_ranking text, Country text, Sector text, Absolute_Growth_Rate number, Compound_Annual_Growth_Rate number, Revenue_2021 number, Revenue_2018 number, Number_of_employees_2021 number, Number_of_employees_2018 number, Founding_Year number)')
conn.commit()

In [39]:
df_ft1000.to_sql('ft1000', conn, if_exists='replace', index = False)

1000

In [43]:
c.execute('''
SELECT * FROM ft1000
LIMIT 10
          ''')

for row in c.fetchall():
    print (row)

('1', 'Tripledot Studios', 'No', 'No', 'UK', 'Leisure & Entertainment', '71,507.8', '794.7', '83,062,822', '115,197', '172', '18', '2016')
('2', 'Marshmallow', 'No', 'No', 'UK', 'Fintech, Financial Services & Insurance', '43,771.2', '659.8', '94,346,611', '201,333', '220', '8', '2016')
('3', 'WeCo', 'No', 'No', 'Italy', 'Manufacturing', '15,048.1', '433.1', '61,882,913', '408,518', '12', '2', '2015')
('4', 'Silverstream Technologies', 'No', 'No', 'UK', 'Professional, Scientific & Technical Services', '14,488.7', '426.4', '17,259,314', '118,306', '48', '8', '2010')
('5', 'Gift & Go', 'No', 'No', 'UK', 'IT & Software', '13,032.6', '408.3', '17,344,060', '123,642', '13', '2', '2017')
('6', 'GT Classic Cars', 'No', 'No', 'France', 'Retail', '11,215.2', '383.7', '14,039,007', '124,072', '9', '1', '2012')
('7', 'illimity Bank', 'No', 'No', 'Italy', 'Fintech, Financial Services & Insurance', '9,251.2', '353.9', '271,184,000', '2,900,000', '725', '138', '2018')
('8', 'Inkitt', 'No', 'No', 'Ger

In [42]:
# df_ft1000.to_csv("ft1000.csv")
# pd.read_csv("ft1000.csv", index_col=0)

# Part II: S&P 500 Companies by Weight

In [7]:
requests.utils.default_headers()

{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

The default `User-Agent` of `requests` is `python-requests/2.31.0`, if website doesn't like traffic from "non-browsers", they will try to block such traffic.

We need to make the request appear like coming from a browser by adding extra `header` parameter.

In [74]:
# headers = {"User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Gecko/20100101 Firefox/45.0"}
# sp500_url = "https://www.slickcharts.com/sp500"
# sp500_page = requests.get(sp500_url, allow_redirects = False, headers = headers).text
# sp500_page = sp500_page.replace(u'\xa0', u'')
# sp500_soup = BeautifulSoup(sp500_page, 'html.parser')

In [8]:
sp500_url = "https://www.slickcharts.com/sp500"
# browser = mechanicalsoup.Browser()
browser = mechanicalsoup.StatefulBrowser()
browser.set_user_agent("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Gecko/20100101 Firefox/45.0")
sp500_page = browser.get(sp500_url)
sp500_soup = sp500_page.soup

In [64]:
sp500_soup = sp500_soup.replace(u'\xa0', u' ')

In [9]:
sp500_soup.title.string

'S&P 500 Companies by Weight'

In [10]:
# column names
sp500_headers = sp500_soup.find_all("th")
sp500_header_col = []
for i in range(len(sp500_headers)):
    sp500_header_col.append(sp500_headers[i].string.replace(u'\xa0', u''))
print(sp500_header_col)

['#', 'Company', 'Symbol', 'Portfolio%', 'Price', 'Chg', '% Chg']


In [11]:
df_sp500 = pd.DataFrame(columns=sp500_header_col, index=pd.RangeIndex(start=0, stop=503, step=1))

In [29]:
sp500_soup.find_all("tr")[1].find_all("td")[4]

<td class="text-nowrap"><img alt="" src="/img/down.gif"/>   188.88</td>

The `get_text(strip=True)` method is used to extract the text content of the tag, removing any leading or trailing whitespace.

In [27]:
sp500 = sp500_soup.find_all("tr")
for i in range(1,len(sp500)-4):
    for j in range(len(sp500_header_col)):
        cell_value = sp500[i].find_all("td")[j].get_text(strip=True)
        df_sp500.iloc[i-1, j] = cell_value
# print(df_sp500)

In [28]:
df_sp500

,#,Company,Symbol,Portfolio%,Price,Chg,% Chg
0,1,Apple Inc.,AAPL,7.30%,188.88,-0.83,(-0.44%)
1,2,Microsoft Corp,MSFT,7.26%,370.94,-5.23,(-1.39%)
2,3,Amazon.com Inc,AMZN,3.44%,143.66,0.83,(0.58%)
3,4,Nvidia Corp,NVDA,3.19%,493.48,-1.32,(-0.27%)
4,5,Alphabet Inc. Class A,GOOGL,2.11%,134.93,-2.00,(-1.46%)
...,...,...,...,...,...,...,...
498,499,"Alaska Air Group, Inc.",ALK,0.01%,36.84,0.64,(1.76%)
499,500,"Solaredge Technologies, Inc.",SEDG,0.01%,75.75,-0.77,(-1.01%)
500,501,"Mohawk Industries, Inc.",MHK,0.01%,85.78,0.29,(0.34%)
501,502,Fox Corporation Class B,FOX,0.01%,28.33,0.19,(0.68%)


The list has 503 symbols due to several companies with two share classes. For example, Google's parent company Alphabet has Class A (GOOGL) and Class C (GOOG) shares in the index.

In [30]:
# save the result in csv file
path = "/content/drive/My Drive/Quantitative-Project/Web Scraping in Python/sp500.csv"
with open(path, 'w', encoding='utf-8-sig') as f:
    df_sp500.to_csv(f)

In [31]:
# from DataFrame to SQL
conn = sqlite3.connect('sp500_database')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS sp500 (Number number, Company text, Symbol text, Portfolio_perc number, Price number, Chg number, Chg_perc number)')
conn.commit()
df_sp500.to_sql('sp500', conn, if_exists='replace', index = False)


503

In [32]:
c.execute('''
SELECT * FROM sp500
LIMIT 10
          ''')

for row in c.fetchall():
    print (row)

('1', 'Apple Inc.', 'AAPL', '7.30%', '188.88', '-0.83', '(-0.44%)')
('2', 'Microsoft Corp', 'MSFT', '7.26%', '370.94', '-5.23', '(-1.39%)')
('3', 'Amazon.com Inc', 'AMZN', '3.44%', '143.66', '0.83', '(0.58%)')
('4', 'Nvidia Corp', 'NVDA', '3.19%', '493.48', '-1.32', '(-0.27%)')
('5', 'Alphabet Inc. Class A', 'GOOGL', '2.11%', '134.93', '-2.00', '(-1.46%)')
('6', 'Meta Platforms, Inc. Class A', 'META', '1.95%', '332.77', '-1.42', '(-0.43%)')
('7', 'Alphabet Inc. Class C', 'GOOG', '1.82%', '136.59', '-2.11', '(-1.52%)')
('8', 'Tesla, Inc.', 'TSLA', '1.77%', '231.04', '-2.55', '(-1.09%)')
('9', 'Berkshire Hathaway Class B', 'BRK.B', '1.72%', '359.75', '-0.11', '(-0.03%)')
('10', 'Unitedhealth Group Incorporated', 'UNH', '1.32%', '533.19', '-6.63', '(-1.23%)')


# Part III: SPX - S&P500 Index Historical Prices

In [ ]:
# headers = {"User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Gecko/20100101 Firefox/45.0"}
# wsj_url = "https://www.wsj.com/market-data/quotes/index/SPX/historical-prices"
# wsj_page = requests.get(wsj_url, allow_redirects = False, headers = headers).text
# wsj_soup = BeautifulSoup(wsj_page, 'html.parser')

In [26]:
wsj_url = "https://www.wsj.com/market-data/quotes/index/SPX/historical-prices"
# browser = mechanicalsoup.Browser()
browser = mechanicalsoup.StatefulBrowser()
browser.set_user_agent("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) Gecko/20100101 Firefox/45.0")
wsj_page = browser.get(wsj_url)
wsj_soup = wsj_page.soup

In [27]:
wsj_soup.title.string

'SPX | S&P 500 Index Historical Prices - WSJ'

In [28]:
# column names
wsj_headers = wsj_soup.find_all("th")
wsj_header_col = []
for i in range(len(wsj_headers)):
    wsj_header_col.append(wsj_headers[i].string)
print(wsj_header_col)

['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE']


In [29]:
df_SPX = pd.DataFrame(columns=wsj_header_col, index=pd.RangeIndex(start=0, stop=63, step=1))

In [34]:
histPrice = wsj_soup.find_all("tr")
for i in range(1,len(histPrice)):
    for j in range(len(wsj_headers)):
        cell_value = histPrice[i].find_all("td")[j].string
        df_SPX.iloc[i-1, j] = cell_value
# print(df_SPX)

In [35]:
df_SPX

,DATE,OPEN,HIGH,LOW,CLOSE
0,11/16/23,4497.08,4511.99,4487.83,4508.24
1,11/15/23,4505.30,4521.17,4495.31,4502.88
2,11/14/23,4458.97,4508.67,4458.97,4495.70
3,11/13/23,4406.66,4421.76,4393.82,4411.55
4,11/10/23,4364.15,4418.03,4353.34,4415.24
...,...,...,...,...,...
58,08/25/23,4389.38,4418.46,4356.29,4405.71
59,08/24/23,4455.16,4458.30,4375.55,4376.31
60,08/23/23,4396.44,4443.18,4396.44,4436.01
61,08/22/23,4415.33,4418.59,4382.77,4387.55


In [31]:
# save the result in csv file
path = "/content/drive/My Drive/Quantitative-Project/Web Scraping in Python/wsj_SPX.csv"
with open(path, 'w', encoding='utf-8-sig') as f:
    df_SPX.to_csv(f)

In [46]:
# from DataFrame to SQL
conn = sqlite3.connect('spx_database')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS spx (Date text, Open number, High number, Low number, Close number)')
conn.commit()
df_SPX.to_sql('spx', conn, if_exists='replace', index = False)

63

In [48]:
c.execute('''
SELECT * FROM spx
LIMIT 10
          ''')

for row in c.fetchall():
    print (row)

('11/16/23', '4497.08', '4511.99', '4487.83', '4508.24')
('11/15/23', '4505.30', '4521.17', '4495.31', '4502.88')
('11/14/23', '4458.97', '4508.67', '4458.97', '4495.70')
('11/13/23', '4406.66', '4421.76', '4393.82', '4411.55')
('11/10/23', '4364.15', '4418.03', '4353.34', '4415.24')
('11/09/23', '4391.41', '4393.40', '4343.94', '4347.35')
('11/08/23', '4384.37', '4391.20', '4359.76', '4382.78')
('11/07/23', '4366.21', '4386.26', '4355.41', '4378.38')
('11/06/23', '4364.27', '4372.21', '4347.53', '4365.98')
('11/03/23', '4334.23', '4373.62', '4334.23', '4358.34')


# Part IV: Summary

***Challenges and How to Resolve them***

1.   Unlike the tutorial website, real-world HTMLs are much more complicated and far less predictable.

- Real-world HTML structures can be intricate and less consistent. Prioritize a flexible approach by dynamically adapting to the structure. Use tools like Selenium or Puppeteer for dynamic content.

2.   Tutorial gives the basic foundation for web scraping but not mention any table scraping tips.

- When scraping tables, identify relevant HTML elements such as `th` for headers and `tr` and `td` for rows. Utilize libraries like BeautifulSoup to streamline table data extraction.

3.   When scraping Wall Street Journal and SlickCharts websites, a `403 Forbidden` status code was received, which means do not have access rights to the content.

- Over come `403 Forbidden` errors by modifying the `User-Agent` in the request headers to mimic a browser. Use a user-agent string that resembles legitimate browser traffic

4.   BeautifulSoup `string` method encounters difficulties in extracting numbers with leading white space like `/xa0`

- Utilize the `get_text(strip=True)` method to extract the text by removing any leading or trailing whitespace.

5.   Websites contain massive amounts of information.

- Define specific goals for scraping and focus on extracting the most valuable information. Streamline the code to target the data relevant to objectives.

***Summary of Results***

1.   For this notebook, I extracted data from three main resources: FT1000 - Europe's fastest growing companies in 2023, SP500 Companies by Weight and SPX Historical Prices for the past quarter.

2.   To enhance the original code, consider leveraging `mechanicalsoup.StatefulBrowser` instead of `mechanicalsoup.Browser` for extended functionality. Additionally, implement error handling using try-except blocks to ensure more robust code execution.


